In [20]:
%run "part01_preprocessing.ipynb"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
 8   Unnamed: 8   0 non-null       float64
dtypes: float64(3), int64(1), object(5)
memory usage: 37.2+ MB


# Data Preparation

In [21]:
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PowerTransformer
from sklearn.decomposition import PCA
from umap.umap_ import UMAP
from sklearn.manifold import TSNE
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

### Content

* **Data Preparation**
    - Normalization
    - Standardization
    - Feature Selection
    - Dealing with Outliers

In [22]:
data_prep = data_client.copy()

In [23]:
data_prep.isna().sum()

GrossRevenueTotal    0
RecencyDays          0
Frequency            0
dtype: int64

In [24]:
#sns.pairplot(data_prep, aspect=1.5);

In [25]:
log_columns = data_prep.skew().sort_values(ascending=False)
log_columns = log_columns.loc[log_columns > 0.75]
log_columns

GrossRevenueTotal    21.585393
Frequency            12.045707
RecencyDays           1.249082
dtype: float64

In [26]:
# The log transformations
for col in log_columns.index:
    data_prep[col] = np.log1p(data_prep[col])

In [27]:
#ss = StandardScaler()
#rs = RobustScaler()
#pt = PowerTransformer()

# MinmmaxScaler provides better resutls mainly because is robbust to outliers
mms = MinMaxScaler()


for col in data_prep.columns:
    data_prep[col] = mms.fit_transform(data_prep[[col]]).squeeze()

In [28]:
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.pipeline import Pipeline

# # The custom NumPy log transformer
# log_transformer = FunctionTransformer(np.log1p)
# estimators = [('log1p', log_transformer), ('mms', MinMaxScaler())]
# pipeline = Pipeline(estimators)
# data_pipe = pipeline.fit_transform(data_client)
# np.allclose(data_pipe, data_prep)

In [29]:
features = ['GrossRevenueTotal', 'RecencyDays', 'Frequency'] # We will use all features
X = data_prep[features].copy() 

In [30]:
#sns.pairplot(X, aspect=1.5);

## Dimensonality Reduction - Variable Spaces

In [31]:
clusters_results = X.copy()

In [17]:
# Dimensionality Reduction using technique UMAP 
umap = UMAP(random_state=3456)
umap_embedding = umap.fit_transform(X)

# X,y axis representation for UMAP
clusters_results['umap_x'] = umap_embedding[:,0]
clusters_results['umap_y'] = umap_embedding[:,1]

In [18]:
# Dimensionality Reduction using technique TSNE
tsne = TSNE(n_components=2, init='pca', learning_rate='auto', n_jobs=-1, random_state=3456)
tsne_embedding = tsne.fit_transform(X)

# X,y axis representation using TSNE
clusters_results['tsne_x'] = tsne_embedding[:,0]
clusters_results['tsne_y'] = tsne_embedding[:,1]

C:\Users\Alysson\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:991: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  warnings.warn(
